# Sentence Embeddings Analysis for Movie ROI Prediction

This notebook uses **transformer-based sentence embeddings** (BERT) to capture the semantic meaning of movie text fields.

Unlike TF-IDF which focuses on word frequency, sentence embeddings:
- Capture **semantic similarity** even with different words
- Understand **context and narrative themes**
- Enable **clustering by meaning** rather than just keywords
- Detect **emotional and thematic patterns** that correlate with ROI

## 1. Setup and Installation

In [ ]:
# Install required packages (run once)
# !pip install sentence-transformers scikit-learn scipy matplotlib seaborn pandas numpy

In [ ]:
import sys
import os

# Add project root to path
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.analysis import EmbeddingsAnalyzer, TextPreprocessor, MovieDataLoader
from src.database.connection import DatabaseConnection

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✓ Imports successful")

## 2. Load Movie Data

In [ ]:
# Initialize database connection
db = DatabaseConnection()
engine = db.get_engine()

# Load movies with text fields
loader = MovieDataLoader(engine)
df = loader.load_movies_with_text(min_budget=100000)

print(f"Loaded {len(df)} movies")
print(f"Average ROI: {df['roi'].mean():.2f}")
print(f"Median ROI: {df['roi'].median():.2f}")

df.head()

## 3. Preprocess Text

Combine text fields with appropriate weights.

In [ ]:
# Initialize preprocessor
preprocessor = TextPreprocessor(
    lowercase=True,
    remove_punctuation=False,  # Keep punctuation for BERT
    remove_numbers=False,
    remove_stopwords=False,  # BERT uses context
    lemmatize=False  # BERT handles morphology
)

# Prepare documents
documents = preprocessor.prepare_movie_documents(
    df,
    title_weight=2,
    overview_weight=3,
    tagline_weight=2,
    genres_weight=2,
    keywords_weight=1
)

print(f"Prepared {len(documents)} documents")
print(f"\nExample document (first 500 chars):\n{documents[0][:500]}")

## 4. Generate Embeddings

Using `all-MiniLM-L6-v2` model:
- Fast and efficient (384 dimensions)
- Good balance between quality and speed
- Trained on 1B+ sentence pairs

In [ ]:
# Initialize embeddings analyzer
embeddings_analyzer = EmbeddingsAnalyzer(
    model_name='all-MiniLM-L6-v2',  # Fast, 384 dimensions
    # model_name='all-mpnet-base-v2',  # Alternative: Higher quality, 768 dimensions
    device='cpu'  # Use CPU (change to 'cuda' if you have compatible GPU with CUDA 11.x or 12.x)
)

# Generate embeddings
embeddings = embeddings_analyzer.encode(
    documents,
    batch_size=32,
    show_progress=True
)

print(f"\nEmbeddings shape: {embeddings.shape}")
print(f"Each movie is represented by a {embeddings.shape[1]}-dimensional vector")

## 5. Correlation Analysis: Which Embedding Dimensions Predict ROI?

Analyze which dimensions of the embedding space correlate with ROI.

In [ ]:
# Compute correlations
dim_correlations = embeddings_analyzer.correlate_embeddings_with_roi(
    df['roi'],
    method='spearman',
    top_dims=30
)

print("Top 20 embedding dimensions correlated with ROI:\n")
print(dim_correlations.head(20))

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))
top_dims = dim_correlations.head(20)

colors = ['green' if x > 0 else 'red' for x in top_dims['correlation']]
ax.barh(range(len(top_dims)), top_dims['correlation'], color=colors, alpha=0.7)
ax.set_yticks(range(len(top_dims)))
ax.set_yticklabels([f"Dim {d}" for d in top_dims['dimension']])
ax.set_xlabel('Correlation with ROI')
ax.set_title('Top 20 Embedding Dimensions by ROI Correlation')
ax.axvline(x=0, color='black', linestyle='--', linewidth=0.8)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

## 6. Semantic Clustering

Group movies by semantic similarity and analyze ROI patterns.

In [ ]:
# Cluster movies into semantic groups
n_clusters = 15
cluster_labels = embeddings_analyzer.cluster_movies(
    n_clusters=n_clusters,
    random_state=42
)

# Analyze ROI by cluster
cluster_stats = embeddings_analyzer.analyze_roi_by_clusters(
    df,
    cluster_labels,
    roi_column='roi'
)

print(f"\nROI Statistics by Semantic Cluster:\n")
print(cluster_stats)

# Visualize
fig, ax = plt.subplots(figsize=(14, 6))
x = range(len(cluster_stats))
ax.bar(x, cluster_stats['roi_mean'], alpha=0.7, label='Mean ROI')
ax.errorbar(x, cluster_stats['roi_mean'], yerr=cluster_stats['roi_std'], 
            fmt='none', color='black', alpha=0.5, capsize=3)
ax.set_xticks(x)
ax.set_xticklabels([f"C{i}\n(n={cluster_stats.iloc[i]['n_movies']})" 
                     for i in range(len(cluster_stats))])
ax.set_ylabel('ROI')
ax.set_title(f'Average ROI by Semantic Cluster ({n_clusters} clusters)')
ax.axhline(y=df['roi'].mean(), color='red', linestyle='--', label='Overall Mean')
ax.legend()
plt.tight_layout()
plt.show()

## 7. Find Representative Movies in Each Cluster

In [ ]:
# Get representative movies (closest to cluster centroid)
representatives = embeddings_analyzer.get_cluster_representative_movies(
    df,
    cluster_labels,
    embeddings,
    n_per_cluster=3
)

# Display representatives for top 5 clusters (by ROI)
top_clusters = cluster_stats.head(5).index

for cluster_id in top_clusters:
    cluster_roi = cluster_stats.loc[cluster_id, 'roi_mean']
    print(f"\n{'='*80}")
    print(f"Cluster {cluster_id} - Mean ROI: {cluster_roi:.2f}")
    print(f"{'='*80}")
    
    rep_movies = representatives[cluster_id]
    for idx, row in rep_movies.iterrows():
        print(f"\n📽️  {row['title']} (ROI: {row['roi']:.2f})")
        print(f"   Genres: {row.get('genres', 'N/A')}")
        if pd.notna(row.get('overview')):
            print(f"   Overview: {row['overview'][:200]}...")

## 8. Visualize Clusters in 2D

Use PCA to reduce embeddings to 2D for visualization.

In [ ]:
# Reduce to 2D using PCA
embeddings_2d = embeddings_analyzer.reduce_dimensions(
    n_components=2,
    embeddings=embeddings
)

# Plot clusters
fig = embeddings_analyzer.plot_clusters_2d(
    embeddings_2d,
    cluster_labels,
    roi_values=df['roi'].values,
    figsize=(16, 7)
)
plt.show()

## 9. Find Similar Movies (Semantic Search)

Discover movies with similar semantic content.

In [ ]:
# Choose a movie to find similar ones
query_movie = "The Dark Knight"  # Change this to any movie in your dataset

# Find index
query_idx = df[df['title'].str.contains(query_movie, case=False, na=False)].index

if len(query_idx) > 0:
    query_idx = query_idx[0]
    
    print(f"Query Movie: {df.iloc[query_idx]['title']}")
    print(f"ROI: {df.iloc[query_idx]['roi']:.2f}")
    print(f"Overview: {df.iloc[query_idx]['overview']}\n")
    
    # Find similar movies
    similar_indices, similarities = embeddings_analyzer.find_similar_movies(
        query_idx,
        top_k=10
    )
    
    print("\nTop 10 Most Similar Movies:\n")
    for i, (idx, sim) in enumerate(zip(similar_indices, similarities), 1):
        movie = df.iloc[idx]
        print(f"{i}. {movie['title']} (Similarity: {sim:.3f}, ROI: {movie['roi']:.2f})")
        print(f"   Genres: {movie.get('genres', 'N/A')}")
        if pd.notna(movie.get('overview')):
            print(f"   Overview: {movie['overview'][:150]}...")
        print()
else:
    print(f"Movie '{query_movie}' not found in dataset")

## 10. Export Embeddings for ML Models

Save embeddings to use as features in prediction models.

In [ ]:
# Create DataFrame with embeddings
embedding_cols = [f'emb_{i}' for i in range(embeddings.shape[1])]
df_embeddings = pd.DataFrame(embeddings, columns=embedding_cols)

# Add movie info and ROI
df_embeddings['movie_id'] = df['id'].values
df_embeddings['title'] = df['title'].values
df_embeddings['roi'] = df['roi'].values
df_embeddings['cluster'] = cluster_labels

# Save to CSV
output_path = '../data/movie_embeddings.csv'
df_embeddings.to_csv(output_path, index=False)

print(f"✓ Saved embeddings to {output_path}")
print(f"  Shape: {df_embeddings.shape}")
print(f"  Columns: {len(embedding_cols)} embedding dimensions + metadata")

## 11. Compare High vs Low ROI Movies Semantically

Analyze semantic differences between successful and unsuccessful movies.

In [ ]:
# Segment by ROI quartiles
roi_quartiles = df['roi'].quantile([0.25, 0.75])
low_roi_mask = df['roi'] <= roi_quartiles[0.25]
high_roi_mask = df['roi'] >= roi_quartiles[0.75]

print(f"Low ROI threshold: {roi_quartiles[0.25]:.2f}")
print(f"High ROI threshold: {roi_quartiles[0.75]:.2f}")
print(f"\nLow ROI movies: {low_roi_mask.sum()}")
print(f"High ROI movies: {high_roi_mask.sum()}")

# Compute average embeddings for each segment
low_roi_centroid = embeddings[low_roi_mask].mean(axis=0)
high_roi_centroid = embeddings[high_roi_mask].mean(axis=0)

# Compute semantic distance
from sklearn.metrics.pairwise import cosine_similarity
semantic_similarity = cosine_similarity(
    low_roi_centroid.reshape(1, -1),
    high_roi_centroid.reshape(1, -1)
)[0, 0]

print(f"\nSemantic similarity between low and high ROI movies: {semantic_similarity:.3f}")
print(f"Semantic distance: {1 - semantic_similarity:.3f}")
print("\n💡 Lower similarity = more semantic difference between segments")

## 12. Conclusions and Next Steps

### Key Findings:

1. **Semantic Clusters**: Movies naturally group by themes/narratives
2. **ROI Patterns**: Certain semantic clusters have higher average ROI
3. **Predictive Dimensions**: Some embedding dimensions correlate with ROI

### How Embeddings Improve ROI Prediction:

- **Beyond Keywords**: Captures meaning, not just word occurrence
- **Narrative Patterns**: Identifies story structures that resonate with audiences
- **Genre-Crossing Themes**: Finds successful patterns across genres
- **Contextual Understanding**: Understands emotional tone and themes

### Next Steps:

1. Use embeddings as features in ML models (Random Forest, XGBoost, Neural Networks)
2. Combine with other features (budget, cast, runtime, etc.)
3. Try different embedding models (mpnet, RoBERTa, etc.)
4. Implement similarity-based prediction (predict ROI from similar movies)
5. Analyze cluster-specific prediction models